Codigo para guardar los landmarks a partir del codigo 1_CreacionDatasetCV2.ipynb

In [29]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.morphology import binary_dilation, binary_erosion, disk
from scipy.ndimage import gaussian_filter
from skimage.filters import gaussian
import os
import shutil
import math

In [30]:
#Load path

data_path = '/home/alanr/Documents/Corazon/Data/'

In [31]:
paths = [f'{data_path}train/mask',
         f'{data_path}val/mask',
         f'{data_path}test/mask']

output_paths = [f'{data_path}train/landmarks',
         f'{data_path}val/landmarks',
         f'{data_path}test/landmarks']

#ULTIMATUM (modify all landmarks into one image)
visual_output_paths = [f'{data_path}train/visual_landmarks',
         f'{data_path}val/visual_landmarks',
         f'{data_path}test/visual_landmarks']

landmark_number = 7

In [32]:
# Clear paths
def delete_directory(directory_path):
    try:
        shutil.rmtree(directory_path)
        print(f"The directory {directory_path} has been deleted.")
    except Exception as e:
        print(f"An error occurred: {e}")

for output_path, visual_path in zip(output_paths, visual_output_paths):
    delete_directory(output_path)
    delete_directory(visual_path)

The directory /home/alanr/Documents/Corazon/Data/train/landmarks has been deleted.
The directory /home/alanr/Documents/Corazon/Data/train/visual_landmarks has been deleted.
The directory /home/alanr/Documents/Corazon/Data/val/landmarks has been deleted.
The directory /home/alanr/Documents/Corazon/Data/val/visual_landmarks has been deleted.
The directory /home/alanr/Documents/Corazon/Data/test/landmarks has been deleted.
The directory /home/alanr/Documents/Corazon/Data/test/visual_landmarks has been deleted.


In [33]:
# Create paths 

for output_path, visual_path in zip(output_paths, visual_output_paths):
    os.makedirs(output_path, exist_ok=True)
    os.makedirs(visual_path, exist_ok=True)


In [34]:
# Define functions

# Sort landmarks
def angle_from_reference(point, reference):
    x, y = point[0] - reference[0], point[1] - reference[1]
    return math.atan2(y, x)

def sort_points_clockwise(points):
    reference_point = min(points, key=lambda p: p[1])
    sorted_points = sorted(points, key=lambda p: angle_from_reference(p, reference_point))
    return sorted_points

def masked_heatmap(outline, landmarks):
    masked_heatmaps = []
    for i in range(len(landmarks)):
        outline = outline / 255.0
        masked_heatmap_i = outline * landmarks[i]  # Create a new array instead of modifying 'outline'
        
        max_val = np.max(landmarks[i])
        masked_heatmap_i = masked_heatmap_i / max_val

        masked_heatmaps.append(masked_heatmap_i)  # Append the new array to the list

    return masked_heatmaps

def extract_landmarks(image):
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    largest_contour = max(contours, key=cv2.contourArea)
    perimeter = cv2.arcLength(largest_contour, closed=True)
    mindis = perimeter / (landmark_number * 2)

    corners = cv2.goodFeaturesToTrack(image, maxCorners=7, qualityLevel=0.01, minDistance=mindis)
    keypoints = [cv2.KeyPoint(x[0][0], x[0][1], 2) for x in corners]
    landmarks = [(int(keypoint.pt[0]), int(keypoint.pt[1])) for keypoint in keypoints]
    sorted_landmarks = sort_points_clockwise(landmarks)
    return sorted_landmarks

In [35]:

incorrect_number_landmark = 0
for input_path, out_path, visual_path in zip(paths, output_paths, visual_output_paths):
    for filename in os.listdir(input_path):
        if filename.endswith('.png') or filename.endswith('.jpg'):
            # Read the image
            image_path = os.path.join(input_path, filename)
            contour_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            

            # Extract landmarks
            landmarks = extract_landmarks(contour_image)

            if len(landmarks) == landmark_number:
                # Create a 3-channel image (for white color)
                contour_image_with_landmarks = cv2.merge([contour_image] * 3)

                # Draw landmarks on the 3-channel image in white color
                for landmark in landmarks:
                    cv2.drawMarker(contour_image_with_landmarks, (landmark[0], landmark[1]), (255, 255, 255), markerType=cv2.MARKER_STAR, markerSize=5)

                # Save the image with white landmarks
                output_path = os.path.join(visual_path, filename)

                cv2.imwrite(output_path, contour_image_with_landmarks)

                  # Generate heatmaps
                heatmaps = []
                for landmark in landmarks:
                    # Use a smaller radius (e.g., 2) for a smaller filled circle
                    radius = 2
                    heatmap = np.zeros_like(contour_image, dtype=np.float32)
                    cv2.circle(heatmap, (landmark[0], landmark[1]), radius, 1, -1)  # Create a filled circle at the landmark position
                    heatmaps.append(heatmap)

                # Apply gaussian filter to each heatmap
                var = 2  # You can adjust the variance according to your requirements
                #Only the point gaussian (landmark)
                gaussian_heatmaps = [gaussian_filter(heatmap, sigma=var) for heatmap in heatmaps]
                #Countour * point
                masked_heatmaps = masked_heatmap(contour_image , gaussian_heatmaps)
                #Countour * point * gaussian
                gaussian_masked_heatmaps = [gaussian_filter(heatmap, sigma=var) for heatmap in masked_heatmaps]
                masked_heatmaps = gaussian_masked_heatmaps

                # Save the heatmaps
                for i, heatmap in enumerate(masked_heatmaps):
                    clean_filename, _ = os.path.splitext(filename)
                    heatmap_filename = f'{clean_filename}_{i}.png'
                    heatmap_path = os.path.join(out_path, heatmap_filename)
                    plt.imsave(heatmap_path, heatmap, cmap='gray')
            else:
                incorrect_number_landmark += 1

print(f'Number of images that doesnt have the correct number of landmarks: {incorrect_number_landmark}')
        

Number of images that doesnt have the correct number of landmarks: 4
